In [1]:
import nltk
import random
import json
import pickle


from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk import flatten
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

import os

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import SGD

#define the lematizer
lemmatizer= WordNetLemmatizer()

class Training:
    def __init__(self):
        #read and load the intent file
        data_file=open("intents.json").read()
        self.intents=json.loads(data_file)["intents"]
        self.ignore_words= list("!@#$%&^?*")
        #process method
        self.process_data()
        
    def process_data(self):
        #fetch pattern and tokenize words 
        
        self.pattern= list(map(lambda x: x["patterns"], self.intents))
      
        self.words= list(map(word_tokenize, flatten(self.pattern)))
       
        
        #fetch tag and store in document with tokenized pattern
        self.classes= flatten([[x["tag"]]*len(y) for x,y in zip(self.intents, self.pattern)])
        
        self.documents=list(map(lambda x ,y: (x, y), self.words, self.classes))
       
        
        #lower and filter the specials symbols in words
        
        self.words= list(map(str.lower, flatten(self.words)))
        self.words= list(filter(lambda x: x not in self.ignore_words, self.words))
        
        #lematize words and sort the class and word lists
        self.words= list(map(lemmatizer.lemmatize, self.words))
        self.words= sorted(list(set(self.words)))
       
        
        self.classes= sorted(list(set(self.classes)))
       
        
    def train_data(self): 
            #initialize and set analiser=words
            cv= CountVectorizer(tokenizer=lambda txt:txt.split(), analyzer="word", stop_words=None)
            
            training=[]
            for doc in self.documents:
                pattern_words= list(map(str.lower, doc[0]))
                pattern_words= " ".join(list(map(lemmatizer.lemmatize, pattern_words)))
                
                #vectorize all words with cv to an one-hot  encoder vector
                vectorize= cv.fit([" ".join(self.words)])
                word_vector= vectorize.transform([pattern_words]).toarray().tolist()[0]
                
                output_row= [0]*len(self.classes)
                
                
                output_row[self.classes.index(doc[1])]=1
                cvop= cv.fit([" ".join(self.classes)])
                out_p= cvop.transform([doc[1]]).toarray().tolist()[0]
                
            
                training.append([word_vector, output_row])
            
            random.shuffle(training)
            training= np.array(training, dtype=object)
            train_x= list(training[:,0])
            train_y= list(training[:,1])
            print(len(train_x[0]))
            return train_x, train_y
         
    def build(self):
        
        train_x, train_y = self.train_data()
        model=Sequential()
        
        model.add(Dense(128, input_shape=( len(train_x[0]) , ) , activation="relu"))
        model.add(Dropout(0.5))
        
        model.add(Dense(64, activation="relu"))
        model.add(Dropout(0.5))
        
        model.add(Dense(len(train_y[0]), activation="softmax"))
        
        sgd= SGD(learning_rate=1e-2, decay=1e-6, momentum=0.9, nesterov=True)
        model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"] )
        
        
        hist= model.fit(np.array(train_x), np.array(train_y), epochs=300, batch_size=10, verbose=1)
        model.save("chatbot_model.h5", hist)
        pickle.dump({"words": self.words, "classes": self.classes, "train_x":train_x, "train_y":train_y}, open("training_data","wb"))

Training().build()  
        

C:\Users\shill\anaconda_sshill\lib\site-packages\sklearn\feature_extraction\text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


97
Epoch 1/300
6/6 [==============================] - 2s 7ms/step - loss: 2.0704 - accuracy: 0.1296
Epoch 2/300
6/6 [==============================] - 0s 6ms/step - loss: 2.0829 - accuracy: 0.0926
Epoch 3/300
6/6 [==============================] - 0s 6ms/step - loss: 2.0859 - accuracy: 0.0556
Epoch 4/300
6/6 [==============================] - 0s 10ms/step - loss: 2.0081 - accuracy: 0.2593
Epoch 5/300
6/6 [==============================] - 0s 8ms/step - loss: 1.9576 - accuracy: 0.2778
Epoch 6/300
6/6 [==============================] - 0s 14ms/step - loss: 1.9719 - accuracy: 0.2593
Epoch 7/300
6/6 [==============================] - 0s 4ms/step - loss: 1.9390 - accuracy: 0.3333
Epoch 8/300
6/6 [==============================] - 0s 4ms/step - loss: 1.8956 - accuracy: 0.2407
Epoch 9/300
6/6 [==============================] - 0s 7ms/step - loss: 1.8211 - accuracy: 0.3148
Epoch 10/300
6/6 [==============================] - 0s 6ms/step - loss: 1.8374 - accuracy: 0.3333
Epoch 11/300
6/6 [======

6/6 [==============================] - 0s 5ms/step - loss: 0.1794 - accuracy: 0.9444
Epoch 85/300
6/6 [==============================] - 0s 6ms/step - loss: 0.1110 - accuracy: 1.0000
Epoch 86/300
6/6 [==============================] - 0s 2ms/step - loss: 0.1050 - accuracy: 1.0000
Epoch 87/300
6/6 [==============================] - 0s 4ms/step - loss: 0.1325 - accuracy: 0.9630
Epoch 88/300
6/6 [==============================] - 0s 7ms/step - loss: 0.1212 - accuracy: 1.0000
Epoch 89/300
6/6 [==============================] - 0s 5ms/step - loss: 0.0822 - accuracy: 1.0000
Epoch 90/300
6/6 [==============================] - 0s 5ms/step - loss: 0.1620 - accuracy: 0.9259
Epoch 91/300
6/6 [==============================] - 0s 6ms/step - loss: 0.1144 - accuracy: 0.9815
Epoch 92/300
6/6 [==============================] - 0s 4ms/step - loss: 0.0729 - accuracy: 1.0000
Epoch 93/300
6/6 [==============================] - 0s 4ms/step - loss: 0.0924 - accuracy: 1.0000
Epoch 94/300
6/6 [===============

6/6 [==============================] - 0s 4ms/step - loss: 0.0485 - accuracy: 1.0000
Epoch 168/300
6/6 [==============================] - 0s 4ms/step - loss: 0.1261 - accuracy: 0.9630
Epoch 169/300
6/6 [==============================] - 0s 5ms/step - loss: 0.0857 - accuracy: 0.9815
Epoch 170/300
6/6 [==============================] - 0s 6ms/step - loss: 0.0568 - accuracy: 0.9815
Epoch 171/300
6/6 [==============================] - 0s 5ms/step - loss: 0.0539 - accuracy: 0.9815
Epoch 172/300
6/6 [==============================] - 0s 4ms/step - loss: 0.0272 - accuracy: 1.0000
Epoch 173/300
6/6 [==============================] - 0s 5ms/step - loss: 0.0431 - accuracy: 0.9815
Epoch 174/300
6/6 [==============================] - 0s 5ms/step - loss: 0.0265 - accuracy: 1.0000
Epoch 175/300
6/6 [==============================] - 0s 4ms/step - loss: 0.0599 - accuracy: 1.0000
Epoch 176/300
6/6 [==============================] - 0s 5ms/step - loss: 0.0280 - accuracy: 1.0000
Epoch 177/300
6/6 [=====

6/6 [==============================] - 0s 5ms/step - loss: 0.0129 - accuracy: 1.0000
Epoch 250/300
6/6 [==============================] - 0s 4ms/step - loss: 0.0221 - accuracy: 1.0000
Epoch 251/300
6/6 [==============================] - 0s 4ms/step - loss: 0.0649 - accuracy: 0.9815
Epoch 252/300
6/6 [==============================] - 0s 5ms/step - loss: 0.0339 - accuracy: 0.9815
Epoch 253/300
6/6 [==============================] - 0s 5ms/step - loss: 0.0059 - accuracy: 1.0000
Epoch 254/300
6/6 [==============================] - 0s 5ms/step - loss: 0.0081 - accuracy: 1.0000
Epoch 255/300
6/6 [==============================] - 0s 4ms/step - loss: 0.0280 - accuracy: 1.0000
Epoch 256/300
6/6 [==============================] - 0s 5ms/step - loss: 0.0275 - accuracy: 0.9815
Epoch 257/300
6/6 [==============================] - 0s 5ms/step - loss: 0.0111 - accuracy: 1.0000
Epoch 258/300
6/6 [==============================] - 0s 4ms/step - loss: 0.0138 - accuracy: 1.0000
Epoch 259/300
6/6 [=====

'mathieu jean luc marc'